## Final Project Submission

* Please fill out:
* John Paul Hernandez Alcala
* Part Time
* Scheduled project review date/time: 
* Instructor name: Eli
* Blog post URL:


### Libraries used

In [23]:
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn
import itertools as it
import requests


### Import Cleaned Budget Data

First we are gonna bring in the movie budget dataframe we obtained from MovieBudgetData.ipynb to use with the OMDb API

In [2]:
%%capture 
%run ./MovieBudgetData.ipynb

In [3]:
%store -r dfmoviebudget
dfmoviebudget.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


### OMDb API Usuage

#### Get API Key to Request Data From OMDb API 

In [32]:
f = open('C:/Users/johnh/.secret/OMDb_API.txt', 'r') #requires valid API_KEY to run
API_KEY = f.read()

Now we search each of the movies through the API

In [5]:
#We would first try with a small sample first to make sure everything will go smoothly for the rest of the data
df2 = dfmoviebudget.copy() #Then progress to all of the data
print(df2.info())
df2.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5126 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5126 non-null   int64 
 1   release_date       5126 non-null   object
 2   movie              5126 non-null   object
 3   production_budget  5126 non-null   int64 
 4   domestic_gross     5126 non-null   int64 
 5   worldwide_gross    5126 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 280.3+ KB
None


,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,425000000,760507625,2776345279
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875
2,3,"Jun 7, 2019",Dark Phoenix,350000000,42762350,149762350
3,4,"May 1, 2015",Avengers: Age of Ultron,330600000,459005868,1403013963
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747


We will use tuples of the moive title and year to deal with our issues of duplicate movie titles in our budget data.

In [6]:
#we will make a list of our movie titles
movieTitles = list(df2.movie)

#Below looks at the release_date column of each movie and splits off the year for each moive title
movieYears = [df2.release_date.iloc[x].split()[2] for x in range(0, len(movieTitles))]

I was not exactly sure how to convert two list to tuples. Here is the solution I uncovered [1] 

In [231]:
movTY = list(zip(movieTitles, movieYears)) #the zip() function helps match the title and year
movTY

[('Avatar', '2009'),
 ('Pirates of the Caribbean: On Stranger Tides', '2011'),
 ('Dark Phoenix', '2019'),
 ('Avengers: Age of Ultron', '2015'),
 ('Star Wars Ep. VIII: The Last Jedi', '2017'),
 ('Star Wars Ep. VII: The Force Awakens', '2015'),
 ('Avengers: Infinity War', '2018'),
 ('Justice League', '2017'),
 ('Spectre', '2015'),
 ('The Dark Knight Rises', '2012'),
 ('Solo: A Star Wars Story', '2018'),
 ('The Lone Ranger', '2013'),
 ('John Carter', '2012'),
 ('Tangled', '2010'),
 ('Spider-Man 3', '2007'),
 ('Captain America: Civil War', '2016'),
 ('Batman v Superman: Dawn of Justice', '2016'),
 ('The Hobbit: An Unexpected Journey', '2012'),
 ('Harry Potter and the Half-Blood Prince', '2009'),
 ('The Hobbit: The Desolation of Smaug', '2013'),
 ('The Hobbit: The Battle of the Five Armies', '2014'),
 ('The Fate of the Furious', '2017'),
 ('Superman Returns', '2006'),
 ('Pirates of the Caribbean: Dead Men Tell No Tales', '2017'),
 ('Quantum of Solace', '2008'),
 ('The Avengers', '2012'),
 (

In [8]:
movTY[0] #how to access one entity in a tuple

('Avatar', '2009')

In [9]:
len(movTY) #Makes sure we still have all our data

5126

Now we take a look at what the data looks like coming from OMDb

In [203]:
title = 'Avatar'
year = '2009'
movieInfo = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+title+'&y='+year).json()
movieInfo

{'Title': 'Avatar',
 'Year': '2009',
 'Rated': 'PG-13',
 'Released': '18 Dec 2009',
 'Runtime': '162 min',
 'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
 'Director': 'James Cameron',
 'Writer': 'James Cameron',
 'Actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang',
 'Plot': 'A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Won 3 Oscars. Another 86 wins & 130 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '82%'},
  {'Source': 'Metacritic', 'Value': '83/100'}],
 'Metascore': '83',
 'imdbRating': '7.8',
 'imdbVotes': '1,129,705',
 'imdbID': 'tt0499549',
 'Type': 'movie',
 'DVD': '10 Feb 2016',
 'BoxOffic

We will use the above as a template for our future dataframe (i.e. keep the keys for use as column names)

In [204]:
movieDetails = movieInfo.copy()

In [265]:
type(movieDetails['Title'])

str

Next, we will define a function that will make sure each key from our template (movie_dict_template) will match each key from the OMDb data we request (incoming_movie)

In [315]:
def moviesDict(movie_dict_template, incoming_movie, OrgMovieTitle):
    '''
    movie_dict_template -- a dictionary or JSON input with keys and values already in place
    incoming_movie -- a dictionary or json input that will be appended to movie_dict_template
    
    
    '''
    #checks to make sure template data has every column as a list to append incoming movie data
    if type(movie_dict_template['Title']) == str: #If title values not list then the rest of columns' values not list
        for key, value in movie_dict_template.items():
            value = [value]
            movie_dict_template[key] = value
    else: #Every column is already a list to add other movies to
        ;
    
    #This fills in fields that might not have been present in incoming movie
    for key, value in movie_dict_template.items(): #Goes through each key in template
        if incoming_movie.get(key): #Checks if template key is present in JSON response
            #Do nothing since this key is present;
            ;
        else:#create this key so our column sizes match for each movie title
            incoming_movie[key] = 'NaN' #This will also fill in movies that do not come up in the API
    
    #This fills in fields that might not have been present in template
    for key, value in incoming_movie.items(): #Goes through each key in incoming movie
        if movie_dict_template.get(key): #Checks if incoming movie key is present in template
            #Do nothing since this key is present;
            ;
        else:#create this key so our column sizes match for each movie title
            incoming_movie[key] = 'NaN'
               
    
    for key, value in movie_dict_template.items():
        for key2, value2 in incoming_movie.items():
            if key == key2:
                #This adds the value from the next movie title to the existing on intial value
                if key == 'Title': #checks for titles that came from API as 'N/A'
                    if value2 != 'NaN':#if title does not have 'N/A' put original title instead
                        movie_dict_template[key].append(OrgMovieTitle)
                    else:#if it does have 'N/A' leave it in there
                        movie_dict_template[key].append(value2)
                        
                else: #Addes values to keys other than the title key.
                    movie_dict_template[key].append(value2)
                    
    return movie_dict_template

Now we see our function in action for each movie title and associated year.

In [237]:
movTY[0]

('Avatar', '2009')

In [13]:
for mov in movTY[1:]: #start at index 1 because we already have first entry
    movieInfo2 = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+mov[0]+'&y='+mov[1]).json()
    movieDetails = moviesDict(movieDetails, movieInfo2, mov[0])

#If error produced it is because API requesting requires subscription to OMDb API

Then we convert the data from movieDetails to a Pandas DataFrame

In [14]:
dfmovieDetails = pd.DataFrame.from_dict(movieDetails)
dfmovieDetails.head(5)

,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,086,714",tt0499549,movie,22 Apr 2010,"$749,700,000",20th Century Fox,N/A,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45,6.6,"465,510",tt1298650,movie,18 Oct 2011,"$241,063,875",Walt Disney Pictures,N/A,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43,5.8,"132,810",tt6565702,movie,03 Sep 2019,N/A,20th Century Fox,N/A,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66,7.3,"704,588",tt2395427,movie,02 Oct 2015,"$429,113,729",Walt Disney Pictures,N/A,True
4,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A


In [15]:
#dfmovieDetails.to_csv('OMDb API Data') #only run if the above has valid API Key and no errors.

## Clean and Export OMDb Data

As you can see, there is a moive which did not get recognized by the OMDb API. We need to clean these out. We noice that the issue is that titles are not being recognized. So let us look at what movies are not being found

In [41]:
df3 = pd.read_csv('./OMDb API Data', index_col=0) #we need index_col = 0 to get rid of index column (i.e. unnamed: 0 column) [2]
df3

,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83.0,7.8,"1,086,714",tt0499549,movie,22 Apr 2010,"$749,700,000",20th Century Fox,NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45.0,6.6,"462,689",tt1298650,movie,18 Oct 2011,"$241,063,875",Walt Disney Pictures,NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43.0,5.8,"132,810",tt6565702,movie,03 Sep 2019,NaN,20th Century Fox,NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66.0,7.3,"704,588",tt2395427,movie,02 Oct 2015,"$429,113,729",Walt Disney Pictures,NaN,True
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's look at all the movie indices that have the title 'NaN'

In [52]:
df3[df3.title.isna()].index

Int64Index([   4,    5,   60,  129,  135,  200,  252,  254,  256,  270,
            ...
            5109, 5110, 5112, 5117, 5119, 5121, 5122, 5123, 5124, 5125],
           dtype='int64', length=746)

Next we will iterate through all the movie titles to obtain a list of movies that came up as 'NaN' from OMDb

In [57]:
NAN_movieTitles = [movieTitles[i] for i in df3[df3.title.isna()].index]
NAN_movieTitles

['Star Wars Ep. VIII: The Last Jedi',
 'Star Wars Ep. VII: The Force Awakens',
 'Prince of Persia: Sands of Time',
 'Cowboys and Aliens',
 'Fast and Furious 6',
 'Men in Black 2',
 'Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part I',
 'X-Men 2',
 'Monster Trucks',
 'Mission: Impossible 2',
 'Fantastic Four: Rise of the Silver Surfer',
 'Star Wars Ep. I: The Phantom Menace',
 'Star Wars Ep. III: Revenge of the Sith',
 'Star Wars Ep. II: Attack of the Clones',
 'Mr. and Mrs. Smith',
 'The Divergent Series: Insurgent',
 'The Divergent Series: Allegiant',
 'The Hangover 3',
 'Seventh Son',
 'Spider-Man: Into The Spider-Verse 3D',
 'Artificial Intelligence: AI',
 'I Now Pronounce You Chuck and Larry',
 'Fantasia 2000 (Theatrical Release)',
 'Battlefield Earth: A Saga of the Year 3000',
 'Arthur et les Minimoys',
 'The Little Prince',
 'AVP: Alien Vs. Predator',
 'Wall Street 2: Money Never Sleeps',
 "Doctor Seuss' The Lorax",
 'Tian jiang xiong s

In [58]:
NAN_movieYears = [movieYears[i] for i in df3[df3.title.isna()].index]
NAN_movieYears

['2017',
 '2015',
 '2010',
 '2011',
 '2013',
 '2002',
 '2011',
 '2010',
 '2003',
 '2017',
 '2000',
 '2007',
 '1999',
 '2005',
 '2002',
 '2005',
 '2015',
 '2016',
 '2013',
 '2015',
 '2018',
 '2001',
 '2007',
 '1999',
 '2000',
 '2007',
 '2016',
 '2004',
 '2010',
 '2012',
 '2015',
 '2016',
 '2009',
 '2005',
 '2009',
 '2017',
 '2008',
 '2016',
 '2008',
 '2016',
 '2015',
 '2003',
 '2011',
 '1981',
 '2002',
 '2006',
 '2013',
 '2004',
 '2014',
 '2016',
 '2005',
 '2009',
 '2009',
 '2015',
 '2001',
 '2007',
 '2013',
 '1982',
 '2010',
 '2005',
 '2005',
 '2005',
 '2006',
 '2005',
 '2016',
 '2017',
 '2011',
 '1999',
 '2015',
 '2007',
 '2013',
 '2002',
 '1997',
 '2000',
 '2019',
 '2011',
 '2016',
 '1979',
 '2009',
 '2005',
 '2017',
 '2010',
 '2000',
 '1993',
 '1996',
 '2009',
 '2014',
 '2006',
 '2015',
 '1983',
 '2005',
 '2017',
 '1999',
 '2004',
 '2010',
 '2004',
 '2011',
 '2017',
 '2011',
 '2003',
 '2006',
 '2018',
 '2007',
 '2010',
 '2011',
 '2008',
 '2015',
 '1998',
 '2003',
 '2002',
 '2011',
 

In [60]:
NAN_movTY = list(zip(NAN_movieTitles, NAN_movieYears)) #the zip() function helps match the title and year
NAN_movTY

[('Star Wars Ep. VIII: The Last Jedi', '2017'),
 ('Star Wars Ep. VII: The Force Awakens', '2015'),
 ('Prince of Persia: Sands of Time', '2010'),
 ('Cowboys and Aliens', '2011'),
 ('Fast and Furious 6', '2013'),
 ('Men in Black 2', '2002'),
 ('Harry Potter and the Deathly Hallows: Part II', '2011'),
 ('Harry Potter and the Deathly Hallows: Part I', '2010'),
 ('X-Men 2', '2003'),
 ('Monster Trucks', '2017'),
 ('Mission: Impossible 2', '2000'),
 ('Fantastic Four: Rise of the Silver Surfer', '2007'),
 ('Star Wars Ep. I: The Phantom Menace', '1999'),
 ('Star Wars Ep. III: Revenge of the Sith', '2005'),
 ('Star Wars Ep. II: Attack of the Clones', '2002'),
 ('Mr. and Mrs. Smith', '2005'),
 ('The Divergent Series: Insurgent', '2015'),
 ('The Divergent Series: Allegiant', '2016'),
 ('The Hangover 3', '2013'),
 ('Seventh Son', '2015'),
 ('Spider-Man: Into The Spider-Verse 3D', '2018'),
 ('Artificial Intelligence: AI', '2001'),
 ('I Now Pronounce You Chuck and Larry', '2007'),
 ('Fantasia 2000 (T

In [81]:
NAN_movTY[0][0][-9:]

'Last Jedi'

In [306]:
NAN_movieDetails = movieInfo.copy()
NAN_movieDetails

{'Title': 'Avatar',
 'Year': '2009',
 'Rated': 'PG-13',
 'Released': '18 Dec 2009',
 'Runtime': '162 min',
 'Genre': 'Action, Adventure, Fantasy, Sci-Fi',
 'Director': 'James Cameron',
 'Writer': 'James Cameron',
 'Actors': 'Sam Worthington, Zoe Saldana, Sigourney Weaver, Stephen Lang',
 'Plot': 'A paraplegic Marine dispatched to the moon Pandora on a unique mission becomes torn between following his orders and protecting the world he feels is his home.',
 'Language': 'English, Spanish',
 'Country': 'USA',
 'Awards': 'Won 3 Oscars. Another 86 wins & 130 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTYwOTEwNjAzMl5BMl5BanBnXkFtZTcwODc5MTUwMw@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '82%'},
  {'Source': 'Metacritic', 'Value': '83/100'},
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A'],
 'Metascore': '83',
 'imdbRating': '7.8',
 'imdbVotes': '1,129,705',
 'imdbID': 'tt0499549

In [308]:
for NAN_mov in NAN_movTY:
    NAN_movieInfo2 = requests.get('http://www.omdbapi.com/?apikey='+API_KEY+'&t='+NAN_mov[0]).json()
    NAN_movieDetails = moviesDict(NAN_movieDetails, NAN_movieInfo2, NAN_mov[0])

#If error produced it is because API requesting requires subscription to OMDb API

In [309]:
NAN_movieDetails

{'Title': ['Avatar',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Monster Trucks',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Monster Trucks',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Mr. and Mrs. Smith',
  'The Divergent Series: Insurgent',
  'N/A',
  'N/A',
  'Seventh Son',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'The Little Prince',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Mei Ren Yu',
  "Michael Jackson's This Is It",
  'N/A',
  'Inkheart',
  'Rock Dog',
  'In the Name of the King: A Dungeon Siege Tale',
  'N/A',
  'N/A',
  'N/A',
  'Paddington',
  'N/A',
  'N/A',
  'Superman II',
  'Blade 2',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Outlander',
  'Taken 3',
  'N/A',
  "Happily N'Ever After",
  'Mr. Nobody',
  'Inchon',
  'N/A',
  'Yours, Mine and Ours',
  'Kicking and Screaming',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'N/A',
  'Baa

In [310]:
df_NAN_movieDetails = pd.DataFrame.from_dict(NAN_movieDetails)
df_NAN_movieDetails

,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83,7.8,"1,129,705",tt0499549,movie,10 Feb 2016,"$760,507,625","Dune, Lightstorm Entertainment, Ingenious Film...",N/A,True
1,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
2,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
3,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
4,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
753,Cavite,2005,Not Rated,12 Mar 2005,80 min,Thriller,"Neill Dela Llana, Ian Gamazon","Neill Dela Llana, Ian Gamazon","Ian Gamazon, Edwin Lagda, Edgar Tancangco, Nor...","Flying back to his home country, an American m...",...,64,6.3,604,tt0428303,movie,02 Dec 2016,N/A,N/A,N/A,True
754,The Mongol King,2005,PG-13,23 Apr 2005,84 min,"Drama, Crime",Anthony Vallone,"Richard Jewell, Robert Morgan, Anthony Vallone","John Considine, Gerry Cooper, Carey Crim, Jess...",While writing a term paper a woman gets involv...,...,N/A,7.8,36,tt0430371,movie,N/A,N/A,N/A,N/A,True
755,Following,1998,R,05 Nov 1999,69 min,"Crime, Mystery, Thriller",Christopher Nolan,Christopher Nolan,"Jeremy Theobald, Alex Haw, Lucy Russell, John ...",A young writer who follows strangers for mater...,...,60,7.5,"87,802",tt0154506,movie,10 Mar 2017,"$48,482",Next Wave Films,N/A,True
756,Return to the Land of Wonders,2004,N/A,27 Apr 2004,88 min,"Documentary, War",Maysoon Pachachi,N/A,N/A,Moving between the political sphere and everyd...,...,60,6.9,37,tt0461832,movie,13 Dec 2005,"$1,338",Arab Film Distribution,N/A,True


In [325]:
df_NAN_movieDetails[df_NAN_movieDetails.Title == 'N/A'].index

Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  11,
            ...
            679, 681, 693, 703, 712, 714, 718, 722, 732, 736],
           dtype='int64', length=222)

In [327]:
NAN_movieTitles2 = [NAN_movieTitles[i-1] for i in df_NAN_movieDetails[df_NAN_movieDetails.Title == 'N/A'].index]
NAN_movieTitles2

['Star Wars Ep. VIII: The Last Jedi',
 'Star Wars Ep. VII: The Force Awakens',
 'Prince of Persia: Sands of Time',
 'Cowboys and Aliens',
 'Fast and Furious 6',
 'Men in Black 2',
 'Harry Potter and the Deathly Hallows: Part II',
 'Harry Potter and the Deathly Hallows: Part I',
 'X-Men 2',
 'Mission: Impossible 2',
 'Fantastic Four: Rise of the Silver Surfer',
 'Star Wars Ep. I: The Phantom Menace',
 'Star Wars Ep. III: Revenge of the Sith',
 'Star Wars Ep. II: Attack of the Clones',
 'Mr. and Mrs. Smith',
 'The Divergent Series: Insurgent',
 'The Divergent Series: Allegiant',
 'The Hangover 3',
 'Seventh Son',
 'Artificial Intelligence: AI',
 'I Now Pronounce You Chuck and Larry',
 'Fantasia 2000 (Theatrical Release)',
 'Battlefield Earth: A Saga of the Year 3000',
 'Arthur et les Minimoys',
 'Wall Street 2: Money Never Sleeps',
 "Doctor Seuss' The Lorax",
 'Mei Ren Yu',
 "Michael Jackson's This Is It",
 'Miss Congeniality 2: Armed and Fabulous',
 'Inkheart',
 'Rock Dog',
 'In the Nam

In [320]:
df_NAN_movieDetails['Title'].values

'Avatar'

We know that since there was index error that all the 'NaN' movies have been identified.

In [17]:
df3 = df3[~df3.title.isna()]
df3.head()

,title,year,rated,released,runtime,genre,director,writer,actors,plot,...,metascore,imdb_rating,imdb_votes,imdb_id,type,dvd,box_office,production,website,response
0,Avatar,2009,PG-13,18 Dec 2009,162 min,"Action, Adventure, Fantasy, Sci-Fi",James Cameron,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",A paraplegic Marine dispatched to the moon Pan...,...,83.0,7.8,"1,086,714",tt0499549,movie,22 Apr 2010,"$749,700,000",20th Century Fox,NaN,True
1,Pirates of the Caribbean: On Stranger Tides,2011,PG-13,20 May 2011,136 min,"Action, Adventure, Fantasy",Rob Marshall,"Ted Elliott (screenplay), Terry Rossio (screen...","Johnny Depp, Penélope Cruz, Geoffrey Rush, Ian...",Jack Sparrow and Barbossa embark on a quest to...,...,45.0,6.6,"462,689",tt1298650,movie,18 Oct 2011,"$241,063,875",Walt Disney Pictures,NaN,True
2,Dark Phoenix,2019,PG-13,07 Jun 2019,113 min,"Action, Adventure, Sci-Fi",Simon Kinberg,Simon Kinberg,"James McAvoy, Michael Fassbender, Jennifer Law...",Jean Grey begins to develop incredible powers ...,...,43.0,5.8,"132,810",tt6565702,movie,03 Sep 2019,NaN,20th Century Fox,NaN,True
3,Avengers: Age of Ultron,2015,PG-13,01 May 2015,141 min,"Action, Adventure, Sci-Fi",Joss Whedon,"Joss Whedon, Stan Lee (based on the Marvel com...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",When Tony Stark and Bruce Banner try to jump-s...,...,66.0,7.3,"704,588",tt2395427,movie,02 Oct 2015,"$429,113,729",Walt Disney Pictures,NaN,True
6,Avengers: Infinity War,2018,PG-13,27 Apr 2018,149 min,"Action, Adventure, Sci-Fi","Anthony Russo, Joe Russo","Christopher Markus (screenplay by), Stephen Mc...","Robert Downey Jr., Chris Hemsworth, Mark Ruffa...",The Avengers and their allies must be willing ...,...,68.0,8.5,"754,875",tt4154756,movie,14 Aug 2018,"$664,987,816",Walt Disney Pictures,NaN,True


In [18]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4380 entries, 0 to 5120
Data columns (total 25 columns):
title          4380 non-null object
year           4380 non-null object
rated          4187 non-null object
released       4336 non-null object
runtime        4341 non-null object
genre          4372 non-null object
director       4316 non-null object
writer         4253 non-null object
actors         4352 non-null object
plot           4294 non-null object
language       4362 non-null object
country        4376 non-null object
awards         3784 non-null object
poster         4260 non-null object
ratings        4375 non-null object
metascore      3841 non-null float64
imdb_rating    4306 non-null float64
imdb_votes     4306 non-null object
imdb_id        4380 non-null object
type           4380 non-null object
dvd            3829 non-null object
box_office     2298 non-null object
production     3827 non-null object
website        10 non-null object
response       4380 non-null 

From above, we can see that we have quite a few NaN values in our dataset. Depending on our analysis, we will drop certain columns. For all analysis, we can drop the 'website' column because it only has 10 valid points and the 'response' column does not relate to the actual movie information

In [19]:
df3.drop(['website', 'response'], axis=1, inplace=True)#inplace being true keeps change

C:\Users\johnh\Anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [20]:
df3.to_csv('ReadyOMDbAPIData')

In [OMDbBudgetDataAnalysis.ipynb](OMDbBudgetDataAnalysis.ipynb), we will use 'ReadyOMDbAPIData' along with R.O.I. data to conduct a further analysis.

## Resources used for development:
1. https://docs.quantifiedcode.com/python-anti-patterns/readability/not_using_zip_to_iterate_over_a_pair_of_lists.html
2. https://stackoverflow.com/questions/36519086/how-to-get-rid-of-unnamed-0-column-in-a-pandas-dataframe
3. http://www.omdbapi.com/